In [5]:
import cv2
import numpy as np
import os

In [ ]:
# Path ke folder gambar dan mask
input_folder = r"dataset/images"
mask_folder = r"dataset/masks_origin"
output_folder = r"segmented_image"

# Pastikan folder output ada
os.makedirs(output_folder, exist_ok=True)

# Loop semua file dalam folder
for filename in os.listdir(input_folder):
    if filename.endswith((".png", ".jpg", ".jpeg")):
        img_path = os.path.join(input_folder, filename)
        mask_path = os.path.join(mask_folder, filename)

        if not os.path.exists(mask_path):
            print(f"Mask untuk {filename} tidak ditemukan, dilewati.")
            continue

        # Baca gambar dan mask
        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Pastikan mask memiliki ukuran yang sama dengan gambar
        mask = cv2.resize(mask, (img.shape[1], img.shape[0]))

        # Konversi mask ke format yang sesuai untuk GrabCut
        mask_gc = np.zeros_like(mask, dtype=np.uint8)
        mask_gc[mask > 200] = cv2.GC_FGD
        mask_gc[mask < 50] = cv2.GC_BGD
        mask_gc[(mask >= 50) & (mask <= 200)] = cv2.GC_PR_FGD

        if (
            np.count_nonzero(mask_gc == cv2.GC_FGD) == 0
            or np.count_nonzero(mask_gc == cv2.GC_BGD) == 0
        ):
            print(f"Mask {filename} tidak valid, dilewati.")
            continue

        # Buat model background dan foreground (dibutuhkan GrabCut)
        bgd_model = np.zeros((1, 65), np.float64)
        fgd_model = np.zeros((1, 65), np.float64)

        # Jalankan GrabCut
        cv2.grabCut(
            img, mask_gc, None, bgd_model, fgd_model, 5, cv2.GC_INIT_WITH_MASK
        )

        # Konversi hasil GrabCut menjadi mask biner
        segmented_mask = np.where(
            (mask_gc == cv2.GC_FGD) | (mask_gc == cv2.GC_PR_FGD), 255, 0
        ).astype("uint8")

        # Terapkan mask ke gambar asli
        segmented_img = cv2.bitwise_and(img, img, mask=segmented_mask)

        # Simpan hasil segmentasi
        output_path = os.path.join(output_folder, "seg_" + filename)
        cv2.imwrite(output_path, segmented_img)

print("Segmentasi selesai, hasil disimpan di:", output_folder)

Mask 013.png tidak valid, dilewati.
Mask 045.png tidak valid, dilewati.
Mask 061.png tidak valid, dilewati.
Mask 062.png tidak valid, dilewati.
Mask 063.png tidak valid, dilewati.
Mask 077.png tidak valid, dilewati.
Mask 086.png tidak valid, dilewati.
Mask 087.png tidak valid, dilewati.
Mask 088.png tidak valid, dilewati.
Mask 156.png tidak valid, dilewati.
Mask 161.png tidak valid, dilewati.
Mask 162.png tidak valid, dilewati.
Mask 163.png tidak valid, dilewati.
Mask 269.png tidak valid, dilewati.
Mask 288.png tidak valid, dilewati.
Mask 305.png tidak valid, dilewati.
Mask 308.png tidak valid, dilewati.
Mask 320.png tidak valid, dilewati.
Mask 324.png tidak valid, dilewati.
Mask 328.png tidak valid, dilewati.
Mask 337.png tidak valid, dilewati.
Segmentasi selesai, hasil disimpan di: segmented_image


In [ ]:
# Path ke folder berisi gambar
input_folder = r"dataset/images"
output_folder = r"segmeted_wth_mask"
# Pastikan folder output ada
os.makedirs(output_folder, exist_ok=True)
# Loop semua file dalam folder
for filename in os.listdir(input_folder):
    if filename.endswith((".png", ".jpg", ".jpeg")):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        # Preprocessing
        blurred = cv2.GaussianBlur(img, (5, 5), 0)
        # Segmentasi dengan thresholding Otsu
        _, segmented = cv2.threshold(
            blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU
        )
        # Simpan hasil segmentasi
        output_path = os.path.join(output_folder, "seg_" + filename)
        cv2.imwrite(output_path, segmented)
print("Segmentasi selesai, hasil disimpan di:", output_folder)

Segmentasi selesai, hasil disimpan di: segmeted_wth_mask
